<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-e6c6fe8dd7979408'
PROJECT = 'qwiklabs-gcp-e6c6fe8dd7979408'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,2126480030009879160,344357
1,8904940584331855459,344191
2,6691862025345277042,338820
3,5934265245228309013,324598
4,1005902328334352553,337637


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13484 examples in the train dataset and 3291 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.000575,False,13,1,37.0,6888635180361964765
1,6.499227,True,13,1,41.0,6637442812569910270
2,6.250105,True,14,1,38.0,1011945037241428753
3,7.352416,False,14,1,40.0,328012383083104805
4,6.124442,True,14,1,39.0,700070369880083266


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13473.000000,13484.000000,13484.000000,13391.000000,1.348400e+04
mean,7.217841,27.353678,1.034782,38.591517,4.372977e+18
std,1.333328,6.131397,0.191932,2.598121,2.797487e+18
min,0.531314,13.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.046873,32.000000,1.000000,40.000000,7.108882e+18
max,13.311511,47.000000,4.000000,47.000000,9.183606e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.000575,False,13,Single(1),37.0,6888635180361964765
1,6.499227,True,13,Single(1),41.0,6637442812569910270
2,6.250105,True,14,Single(1),38.0,1011945037241428753
3,7.352416,False,14,Single(1),40.0,328012383083104805
4,6.124442,True,14,Single(1),39.0,700070369880083266


In [11]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13479,5.000084,Unknown,46,Single(1),37.0,2768780346491825728
13480,2.160530,Unknown,46,Single(1),34.0,4329667052416032880
13481,7.500126,Unknown,46,Single(1),36.0,1305143018446161857
13482,6.433089,Unknown,47,Single(1),37.0,5960947298512945432
13483,3.562670,Unknown,47,Multiple(2+),31.0,3572456083892950068


In [12]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26764.000000,26764.000000,26764.000000,2.676400e+04
mean,7.219480,27.351368,38.597444,4.370849e+18
std,1.330514,6.130979,2.580432,2.796482e+18
min,0.531314,13.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.046873,32.000000,40.000000,7.108882e+18
max,13.311511,47.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [14]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6522 eval.csv
  26764 train.csv
  33286 total
==> eval.csv <==
7.31273323054,False,13,Single(1),40.0,1639186255933990135
6.62930021834,True,14,Single(1),38.0,7517141034410775575
6.3118345610599995,False,14,Single(1),34.0,7517141034410775575
3.75006307662,True,14,Twins(2),32.0,6392072535155213407
7.06361087448,False,14,Single(1),37.0,8904940584331855459
8.437090766739999,False,14,Single(1),39.0,6365946696709051755
7.5728786996999995,True,14,Single(1),38.0,6141045177192779423
7.50012615324,True,14,Single(1),41.0,270792696282171059
5.1918862700999995,True,14,Single(1),40.0,6910174677251748583
9.49971886958,False,14,Single(1),36.0,3182182455926341111

==> train.csv <==
8.000575487979999,False,13,Single(1),37.0,6888635180361964765
6.4992274837599995,True,13,Single(1),41.0,6637442812569910270
6.2501051276999995,True,14,Single(1),38.0,1011945037241428753
7.3524164377,False,14,Single(1),40.0,328012383083104805
6.1244416383599996,True,14,Single(1),39.0,700070369880083266
7.3744626639,False,1

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License